# Lab 1

In [1]:
from cplex import Cplex

In [4]:
from docplex.mp.model import Model
# Create model
mdl = Model(name="DemoDocplex")
# Variables
x = mdl.continuous_var(name="x", lb=0)
y = mdl.continuous_var(name="y", lb=0)
# Objective: Maximize x + y
mdl.maximize(x + y)
# Constraints
mdl.add_constraint(x + 2 * y <= 4, "c1")
mdl.add_constraint(4 * x + 3 * y <= 12, "c2")
# Solve
sol = mdl.solve(log_output=True)
# Output
if sol:
    print(f"objective value: {mdl.objective_value}")
    mdl.print_solution()
else:
    print("No solution found.")

Version identifier: 22.1.1.0 | 2022-11-28 | 9160aff4d
CPXPARAM_Read_DataCheck                          1
Tried aggregator 1 time.
No LP presolve or aggregator reductions.
Presolve time = 0.00 sec. (0.00 ticks)

Iteration log . . .
Iteration:     1   Dual infeasibility =             0.000000
Iteration:     2   Dual objective     =             3.200000
objective value: 3.2
objective: 3.200
status: OPTIMAL_SOLUTION(2)
  x=2.400
  y=0.800
